## Demo PyHeat Val_direct

Ceci est une notebook de démo pour la récupération de valeur de température d'après la solution numérique des équations hydrothermiques.

In [1]:
from pyheatmy import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# paremètres de l'expérience
t_debut = (2011, 8, 1)
t_fin = (2011, 8, 31, 23, 59, 59)
dt = 15*60 # pas de temps en (s)

T_riv_amp = 4
T_riv_offset = 20
P_T_riv = 72*4*dt

T_aq_amp = 0
T_aq_offset = 14
P_T_aq = 9999 # à mettre dans le init

dH_amp = 0
dH_offset = 1
P_dh = -9999 #14*24*4*dt

depth_sensors = [.1, .2, .3, .4]

moinslog10K = 4
n = .1
lambda_s = 2
rho_cs = 4e6


In [ ]:
# définition de l'expérience 

"""Times series"""
time_series_dict = {
    "offset" : 0,
    "depth_sensors":depth_sensors,
	"param_time_dates": [t_debut, t_fin, dt], 
    "param_dH_signal": [dH_amp, P_dh, dH_offset], #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"param_T_riv_signal": [T_riv_amp, P_T_riv, T_riv_offset],
    "param_T_aq_signal": [T_aq_amp, P_T_aq, T_aq_offset],
    "sigma_meas_P": 0.05,
    "sigma_meas_T": 0.1, #float
}
time_series = Time_series.from_dict(time_series_dict)

"""Colonne"""
col_dict = {
	"river_bed": 1., 
    "depth_sensors": depth_sensors, #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"offset": .0,
    "dH_measures": time_series._T_riv_dH_measures,
    "T_measures": time_series._T_Shaft_measures,
    "sigma_meas_P": time_series._sigma_P, #float
    "sigma_meas_T": time_series._sigma_T, #float
}
colonne_exp = Column.from_dict(col_dict)

"""Layer"""
monolayer_dict = {
    name: "sable",
    zLow: depth_sensors[-1],
    moinslog10K: moinslog10K,
    n: n,
    lambda_s: lambda_s,
    rhos_cs: rho_cs
}
monolayer = Layer.from_dict(monolayer_dict)

In [2]:
val_dir_dict = {
    "depth_sensors": [.15, .25, .4], #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"offset": .05,
    "dH_measures": list(zip(times,list(zip(-.01*np.cos(np.linspace(0,2*np.pi,500))+.03, temps[:,0])))),
    "moinslog10K": 4,
    "lambda_m": 1, #a modifier
    "rho_c_m": 1, #a modifier
    "rho_c_w": 1 #a modifier
}

col_analy = valdirect.from_dict(val_dir_dict)

TypeError: valdirect.__init__() got an unexpected keyword argument 'depth_sensors'

In [ ]:
ncell =100

col_analy.compute_temp_general(ncell, 24, 1, 1)
col_analy.compute_temp_cond(ncell, 24 , 1, 1)

In [ ]:
col.get_loc(100)